In [13]:
%pip install openai -q
from openai import OpenAI
%pip install diptest
import numpy as np
from scipy.stats import ttest_ind
from diptest import diptest
import matplotlib.pyplot as plt
from scipy.stats import norm  # For Gaussian fitting
import seaborn as sns
import json
import io
import base64

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
!pip3 install requests
import requests
import sqlite3
import json
from bs4 import BeautifulSoup
import random
def getData(url):
    response = requests.get(url)
    #convert to text string and return 
    return response.text


url = 'https://perfmon-events.intel.com/ahybrid.html'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 gecko/20100101 Firefox/102.0"}
html_doc = getData(url)
soup = BeautifulSoup(html_doc,'html.parser')
#print(soup)
#2
#table=soup.find('table',{'class':'sortable wikitable'})
table=soup.find('table',{'id':'event_table'})
#print(table)
table_val=[]
rows=table.find_all('tr')[2:]

print(rows[1].find_all('td')[2].text)
event_table={}
for r in range(5,len(rows)):
    td_tag=rows[r].find_all('td')
    if(td_tag):
        key = td_tag[0].text.strip()  # key
        value = td_tag[2].text.split(" ")[:2]  # Extract first two words from the 3rd column
        event_table[key] = {
            'config': value,  # Add the configuration field
            'description': td_tag[1].text.strip()  # Extract the description from the 2nd column
        }


 

IA32_FIXED_CTR1  PEBS:[NonPreciseEventingIP]  Architectural, Fixed, Speculative


In [15]:
import openai
import tempfile

class GPT:
   
    def __init__(self):
        # Initialize the client with your API key
        self.api_key = GPT.read_api_key()
        self.client=OpenAI(api_key=self.api_key)
        self.event_table=event_table
        self.tools=[
                {
                    "type": "function",
                    "function":{
                    "name": "perform_bimodal_test",
                    "description": "This function performs a diptest on the  HPC's values given by the user and assistant'll return the stats value if p value <0.05 then it is bimodal",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "filepath": {
                                "type": "string",
                                "description": "perfmon event counter filepath eg filepath.txt"
                            }
                            },
                        "required": ["filepath"]
                    }
                }
                },
                {
                    "type": "function",
                    "function": {
                        "name": "plot_a_graph",
                        "description": "This function reads data from a file and plots a graph of the data. that show it is bimodal or what",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "filepath": {
                                    "type": "string",
                                    "description": "Path to the data file (e.g., filepath.txt)."
                                }
                            },
                            "required": ["filepath"]
                        }
                    }
                }
            ]
       
    @staticmethod
    def read_api_key():
        # Read the API key from the text file
        try:
            with open('adap_key', 'r') as file: # here your key location 
                api_key = file.read().strip()
            return api_key
        except FileNotFoundError:
            raise Exception("API key file 'adap_key' not found.")
        except Exception as e:
            raise Exception(f"An error occurred while reading the API key: {e}")
        
    def get_gpt_response(self,messages):

        try:
            # Call the OpenAI ChatCompletion API
           
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                tools=self.tools,
                tool_choice="auto",
            )
            # Extract and return the assistant's response
            return  response
        except Exception as e:
            raise Exception(f"An error occurred while getting GPT response: {e}")
    def upload_file(self, filepath):
        with open(filepath, 'rb') as file:
            response = self.client.files.create(file=file, purpose="assistants")
        return response.id
    def read_data_from_file(file_path):
        """Reads numeric data from a file, assuming one value per line."""
        data = []

        with open(file_path, 'r') as file:
            for line in file:
                data.extend([float(value) for value in line.strip().split()])
        return data[:72000]

    def perform_bimodal_test(filepath):
        """Performs the Dip Test on the data."""
        try:
            # Read data from the file
            data = GPT.read_data_from_file(filepath)
            # Convert list to numpy array
            data_array = np.array(data)
            #print(data_array)
            # Perform the Dip Test
            dip_stat, p_value = diptest(data_array)
            #print(f"Dip stat: {dip_stat}, P-value: {p_value}")  # Debug results

            return f"dip_stat: {dip_stat}, p_value: {p_value}"
        except Exception as e:
            return {"error": str(e)}
    def plot_a_graph(filepath):
        try:
            data=GPT.read_data_from_file(filepath)
            plt.figure(figsize=(8, 6))
            sns.histplot(data, kde=True, bins=30, color="blue", stat="density", label="Histogram with KDE")

            # Add title and labels
            plt.title("Check for Bimodality in Data", fontsize=16)
            plt.xlabel("Data Values", fontsize=12)
            plt.ylabel("Density", fontsize=12)
            plt.legend()
            plt.grid(alpha=0.3)
            # Save the plot to a temporary file
            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".png")
            plt.savefig(temp_file.name, format="png", bbox_inches="tight")
            
            # Close the plot
            plt.close()
            
            # Return the file path
            return {"image_path": temp_file.name}
            #plt.show()
            '''buffer = io.BytesIO()
            plt.savefig(buffer, format="png")
            buffer.seek(0)

            plt.close()

            # Encode the image as base64
            img_base64 = base64.b64encode(buffer.read()).decode("utf-8")
            buffer.close()
            return img_base64'''
        except Exception as e:
            return {"error": str(e)}

   

In [16]:
# Prepare your messages
messages=[{'role': 'system', 'content': 'You are an assistant ,that gives a perfmon event counter from the site https://perfmon-events.intel.com/ to detect attack by given the decription of the attack and machine model'},
      ]
'''
    {'role':'user','content':'I am performing a matrix multiplication of a 500x500 size matrix. Can you suggest list of  counters name for raptor lake system from intel perfmon that could potentially be used to detect attack on my victim code? '}
        ]'''
print(f"system :{messages[0]['content']}")

system :You are an assistant ,that gives a perfmon event counter from the site https://perfmon-events.intel.com/ to detect attack by given the decription of the attack and machine model


In [13]:
messages=[{
            "role": "system",
            "content": "You are an assistant ,that gives a perfmon event counter to detect attack by given the decription of the attack"
        },
        {
            "role": "user",
            "content": "I am performing a matrix multiplication of a 500x500 size matrix. Can you suggest set of  counters name for raptor lake system from intel perfmon that could potentially be used to detect meltdown attack on my victim code?"
        },
        {
            "role": "assistant",
            "content": "To detect a Meltdown attack on your victim code using Intel Perfmon counters for the Raptor Lake system, you can consider the following set of counters:\n\n1. **INST_RETIRED.ANY**: Counts the number of instructions retired.\n2. **CPU_CLK_UNHALTED.THREAD**: Counts the number of core clock cycles when the thread is running.\n3. **MEM_INST_RETIRED.ALL_LOADS**: Counts the number of retired instructions that are data loads.\n4. **MEM_INST_RETIRED.ALL_STORES**: Counts the number of retired instructions that are data stores.\n5. **L2_TRANS.L2_WB**: Counts L2 cache writebacks.\n6. **L2_RQSTS.ALL_CODE_RD**: Counts all code reads.\n7. **ARITH.INST_RETIRED**: Counts the number of retired instructions that are arithmetic instructions.\n\nThese counters can provide insights into the behavior of the code and potential anomalies that could indicate a Meltdown attack. By monitoring these counters during the matrix multiplication operation, you can analyze if there are any sudden spikes or unusual patterns that might signal a Meltdown attack.\n\nWould you like to proceed with monitoring these counters using a performance monitoring event counter file (perfmon event counter)?"
        }]

In [17]:
messages

[{'role': 'system',
  'content': 'You are an assistant ,that gives a perfmon event counter from the site https://perfmon-events.intel.com/ to detect attack by given the decription of the attack and machine model'}]

In [18]:
def append_to_json(file_path, new_data):
    try:
        # Read the existing data from the file
        try:
            with open(file_path, "r") as file:
                existing_data = json.load(file)  # Load existing JSON data
        except (FileNotFoundError, json.JSONDecodeError):
            existing_data = []  # If file doesn't exist or is empty, start with an empty list

        # Append the new data
        existing_data.append(new_data)

        # Write the updated data back to the file
        with open(file_path, "w") as file:
            json.dump(existing_data, file, indent=4)  # Save the updated JSON
        print("Messages successfully appended to all.json.")
    except Exception as e:
        print(f"Error appending to JSON file: {e}")

In [19]:
# Usage



try:
   
    # Initialize your custom client with the API key
    allin_path = "all.json"
    client = GPT()
   #messages=[{'role': 'system', 'content': 'You are an assistant ,that gives a perfmon event counter to detect attack by given the decription of the attack'}]
    
    t=1
    while 1:
        print("user :")
        user_reply=input().strip()
        # Exit the loop if the user types "exit" or "quit"
        if user_reply.lower() in {"exit", "quit"}:
            append_to_json(allin_path, messages)
            print("Exiting the chat. Goodbye!")
            break
        messages.append({'role': 'user','content':f"{user_reply}"})
        print(user_reply)
        #Get the GPT response using your custom client
        response = client.get_gpt_response(messages)

        tool_calls = response.choices[0].message.tool_calls
        #print(tool_calls)
        available_functions = {
                "perform_bimodal_test": GPT.perform_bimodal_test,
                "plot_a graph":GPT.plot_a_graph,
            }
        if(tool_calls and "perform a diptest" in user_reply.lower()) :
            '''gpt_response = {
            "role": response.choices[0].message.role,
            "content": response.choices[0].message
            }'''
            messages.append(response.choices[0].message)
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                
                if function_args.get("filepath").endswith(".txt"):
                    function_response = function_to_call(filepath=function_args.get("filepath"))
                    file_id=GPT.upload_file(function_args.get("filepath"))
                    
                    # Append the function response back to the conversation
                    messages.append(
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": function_response,
                        }
                    )

            response = client.get_gpt_response(messages)
            messages.append({'role':'assistant','content':f"{response.choices[0].message.content}"})
            print("GPT Response:", response.choices[0].message.content)
        elif(tool_calls and  "plot a graph" in user_reply.lower()):
            '''gpt_response = {
            "role": response.choices[0].message.role,
            "content": response.choices[0].message
            }'''
            messages.append(response.choices[0].message)
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                if function_args.get("filepath").endswith(".txt"):
                    function_response = function_to_call(filepath=function_args.get("filepath"))
                    # Append the function response back to the conversation
                    messages.append(
                        {
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "name": function_name,
                            "content": function_response,
                        }
                    )
            #img_html = f'<img src="data:image/png;base64,{function_response}" alt="Graph">'
            #print(f"Here is your graph:\n{img_html}")
            response = client.get_gpt_response(messages)
            messages.append({'role':'assistant','content':f"{response.choices[0].message.content}"})
            print("GPT Response:", response.choices[0].message.content)
        else:
            role=response.choices[0].message.role
            content=response.choices[0].message.content
            messages.append(
                {
                    "role":role,
                    "content":content
                }
            )
            print("GPT Response:", response.choices[0].message.content)

        
except Exception as e:
    print("Error:", e)


user :
I am performing RSA encryption and decryption.Can you suggest set of  counters name for alder lake system from intel perfmon that could potentially be used to detect speculative execution  with Croos-Modifying Code Machine Clear on my system..
GPT Response: To detect speculative execution vulnerabilities such as "Cross-Modifying Code" on an Alder Lake system, you can refer to Intel's performance monitoring counters. Here are some potential perfmon event counters that may be helpful in this context:

1. **IDQ_UOPS_NOT_DELIVERED.CORE**: Counts the number of Uops not delivered by the front end.

2. **INT_MISC.RECOVERY_CYCLES**: Counts the number of cycles that the machine is recovering from previous mispredictions or interruptions.

3. **CYCLES_ACTIVITY.CYCLES_NO_EXECUTE**: Counts the number of cycles where no instructions are executed.

4. **MACHINE_CLEARS.SMC**: Counts the occurrences of Self-Modifying Code, which might trigger speculative execution vulnerabilities.

5. **MACHINE

In [136]:
json.loads(response.choices[0].message.tool_calls[0].function.arguments).get('filepath')

'perfw_CPU_CLK_UNHALTED_THREAD_P.txt'

In [142]:
print(GPT.perform_unimodal_test(filepath=json.loads(response.choices[0].message.tool_calls[0].function.arguments).get('filepath')))


dip_stat: 0.03286805555555555, p_value: 0.0


In [10]:
messages

[{'role': 'system',
  'content': 'You are an assistant ,that gives a perfmon event counter from the site https://perfmon-events.intel.com/ to detect attack by given the decription of the attack and machine model'},
 {'role': 'user',
  'content': 'I am performing RSA encryption and decryption.Can you suggest set of  counters name for alder lake system from intel perfmon that could potentially be used to detect speculative execution  with Croos-Modifying Code Machine Clear on my system.'}]